Packages:

In [2]:
import glob
import os
import warnings


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split

from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
importlib.reload(help_files._1_visuals_script)
# importlib.reload(help_files._01_load_data)
import  help_files._1_visuals_script  as pauls_vs
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [3]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())

In [4]:
# Save the filtered dataframes to CSV files
dataframes = ["df_end"]
file_names = ["df_end.csv"]
 
# Load the data from _01_load_data
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
df_end = dataframes[0]


In [5]:
df_end.dtypes

study_id        int64
severity      float64
condition       int64
level          object
series_id     float64
x             float64
y             float64
image_path     object
dtype: object

first try 

In [6]:
# Randomly sample 100 rows from random_samples_combined
random_samples_test_check = df_end.sample(n=100, random_state=RSEED)
random_samples_test_check


,study_id,severity,condition,level,series_id,x,y,image_path
1801,2543001868,1.0,0,l5/s1,7.878829e+08,282.233215,309.144876,data/train_images_origin/2543001868/1289783254...
1190,1698156042,2.0,0,l5/s1,1.349877e+09,178.912134,213.389121,data/train_images_origin/1698156042/3800217841...
1817,2568819355,0.0,0,l5/s1,6.402851e+08,360.224323,568.235013,data/train_images_origin/2568819355/4223104020...
251,358286753,2.0,0,l5/s1,1.392935e+09,133.461942,246.570393,data/train_images_origin/358286753/4144723093/...
2505,3515641631,1.0,0,l5/s1,3.168174e+09,408.250460,550.081594,data/train_images_origin/3515641631/4119560068...
...,...,...,...,...,...,...,...,...
416,607436629,0.0,0,l5/s1,4.274491e+09,208.310903,246.131231,data/train_images_origin/607436629/493360101/5...
2631,3742728457,0.0,0,l5/s1,2.064758e+09,169.131790,266.956346,data/train_images_origin/3742728457/4105399347...
2291,3239069834,1.0,0,l5/s1,2.276407e+09,233.190776,394.465409,data/train_images_origin/3239069834/2352610120...
174,286903519,0.0,0,l5/s1,1.527286e+09,229.419524,316.628781,data/train_images_origin/286903519/1921917205/...


Restore parameters and metrtics from mlflow

I should take prdicted probabilities and not predicted classes: output scores (logits) are converted into probabilities using the softmax function

In [7]:
import mlflow
import mlflow.pytorch
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.models as models
from torchvision import transforms
import pydicom
import cv2
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

# Set the experiment name
experiment_name = "Resnet50_MRI_Classification"
mlflow.set_experiment(experiment_name)

# Specify the run ID of the logged model
run_id = "f390913c59d642329c86d0f52b943062"  # Replace with your actual run ID
model_uri = f"runs:/{run_id}/model"

# Load the model
model = mlflow.pytorch.load_model(model_uri)
model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))  # Move model to appropriate device
model.eval()  # Set the model to evaluation mode

# Prepare data
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class MRIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.data['severity'] = self.data['severity'].astype(int)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = row['image_path']
        label = row['severity']
        dicom_image = pydicom.dcmread(image_path)
        image = dicom_image.pixel_array.astype(float)
        image = (image / image.max() * 255).astype('uint8')
        if len(image.shape) == 2:  # Grayscale
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        image_tensor = self.transform(image) if self.transform else torch.from_numpy(image).permute(2, 0, 1)
        return image_tensor, torch.tensor(label).long()

    def __len__(self):
        return len(self.data)

predicting probabilities for test data

In [8]:
test_data = random_samples_test_check

test_dataset = MRIDataset(data=test_data, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Inference loop with probability extraction
results = []
probabilities_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)  # Ensure labels are also on the correct device
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Calculate class probabilities
        _, predicted_classes = torch.max(outputs, 1)
        
        # Append predictions and probabilities
        results.append(predicted_classes.item())
        probabilities_list.append(probabilities.cpu().numpy())



Using device: cpu


In [9]:
# Print predicted classes and their probabilities
for i, (pred, probs) in enumerate(zip(results, probabilities_list)):
    print(f"Test image {i}: Predicted class {pred}, Probabilities: {probs}")

# Additional code to plot the confusion matrix can stay as-is:
true_labels = []
predicted_labels = []

for images, labels in test_loader:
    images = images.to(device)
    true_labels.extend(labels.numpy())

    with torch.no_grad():
        outputs = model(images)
        _, predicted_classes = torch.max(outputs, 1)
        predicted_labels.extend(predicted_classes.cpu().numpy())

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

print("True Labels Unique Values:", np.unique(true_labels))
print("Predicted Labels Unique Values:", np.unique(predicted_labels))


Test image 0: Predicted class 2, Probabilities: [[0.15330388 0.37742668 0.46926945]]
Test image 1: Predicted class 2, Probabilities: [[0.13040946 0.37752903 0.49206153]]
Test image 2: Predicted class 1, Probabilities: [[0.23696716 0.44675255 0.31628034]]
Test image 3: Predicted class 0, Probabilities: [[0.44080806 0.22888544 0.33030653]]
Test image 4: Predicted class 0, Probabilities: [[0.47634664 0.0746745  0.44897893]]
Test image 5: Predicted class 2, Probabilities: [[0.3311112  0.32648456 0.3424043 ]]
Test image 6: Predicted class 0, Probabilities: [[0.4435912  0.22052014 0.33588868]]
Test image 7: Predicted class 2, Probabilities: [[0.14793454 0.2277063  0.6243592 ]]
Test image 8: Predicted class 2, Probabilities: [[0.09944285 0.4276044  0.4729527 ]]
Test image 9: Predicted class 1, Probabilities: [[0.11186356 0.544449   0.3436875 ]]
Test image 10: Predicted class 1, Probabilities: [[0.14050353 0.43831402 0.4211824 ]]
Test image 11: Predicted class 0, Probabilities: [[0.45977855 0.

In [10]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Create a DataFrame from the confusion matrix
conf_matrix_df = pd.DataFrame(
    conf_matrix, 
    index=[f"Actual {i}" for i in range(len(conf_matrix))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_df.style.background_gradient(cmap='Blues')


          Predicted 0  Predicted 1  Predicted 2
Actual 0            5           11           22
Actual 1            6            3           22
Actual 2            2            8           21


,Predicted 0,Predicted 1,Predicted 2
Actual 0,5,11,22
Actual 1,6,3,22
Actual 2,2,8,21


In [11]:
# Convert the list of probabilities to a numpy array for easier manipulation
probabilities_array = np.vstack(probabilities_list)

# Calculate the average probability for each class
average_probabilities = np.mean(probabilities_array, axis=0)

# Print the average probabilities
for i, avg_prob in enumerate(average_probabilities):
    print(f"Average probability for class {i}: {avg_prob:.4f}")

Average probability for class 0: 0.2240
Average probability for class 1: 0.3211
Average probability for class 2: 0.4550


In [12]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
# Calculate precision, recall, and accuracy using mean_probabilities_df
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")


Precision: 0.2886
Recall: 0.2900
Accuracy: 0.2900
